In [1]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [2]:
clslist = list(set(train['cls_2'].tolist()))
len(clslist)

6

In [3]:
from keras.utils import to_categorical

clsdict = {}
num = 0
for i in clslist:
    clsdict[num] = i
    num += 1

one_hot_labels = to_categorical(list(clsdict.keys()), num_classes=6)

clsdict_inv = dict(zip(clsdict.values(), clsdict.keys()))

In [4]:
#train['vector_cls'] = None
train['vector_cls'] = train['cls_2'].apply(lambda x : one_hot_labels[clsdict_inv[x]])

In [6]:
import jieba
from types import MethodType, FunctionType
import re
jieba.load_userdict("newdict.txt")

def clean_txt(raw):
    fil = re.compile(r"[^0-9a-zA-Z\u4e00-\u9fa5]+")
    return fil.sub(' ', raw)

def seg(sentence, sw, apply=None):
    if isinstance(apply, FunctionType) or isinstance(apply, MethodType):
        sentence = apply(sentence)
    return_list = []
    for i in jieba.cut(sentence):
        if i.strip() and i not in sw:
            return_list.append(str(i))
    return return_list

def stop_words():
    with open('stopwords.txt', 'r', encoding='utf-8') as swf:
        return [line.strip() for line in swf]

def save2file(dataFrame, sFilePath):
    fpOut = open(sFilePath,'w')
    
    for index, row in dataFrame.iterrows():
        if str(row[1]).strip() == "":
            print("Warning：Meet a empty-content record at line:" + str(index+1))
            continue
        str_info = str(row[0]) + " " + str(row[1]) +"\n"
        fpOut.write(str_info)
    
    fpOut.close

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\康文轩\AppData\Local\Temp\jieba.cache
Loading model cost 1.062 seconds.
Prefix dict has been built successfully.


In [7]:
train['name'] = train['name'].apply(lambda x : seg(x, stop_words(), apply=clean_txt))
#train_data['name'] = train_data['name'].parallel_apply(lambda x : [i for i in x])


In [8]:
def bigram(l):
    newl = l.copy()
    #newl = []
    for i in range(len(l)-1):
        newl.append(l[i]+l[i+1])
    return newl
bigram(['兖州', '特曲', '品香'])

['兖州', '特曲', '品香', '兖州特曲', '特曲品香']

In [9]:
train['name'] = train['name'].apply(lambda x : bigram(x))

In [10]:
from itertools import chain
all_words = list(chain.from_iterable(train['name']))
# 创建分词数据框
corpus = pd.DataFrame(all_words, columns=['word'])
corpus['cnt'] = 1

# 分组统计
g = corpus.groupby(['word']).agg({'cnt': 'count'}).sort_values('cnt', ascending=False)

g

,cnt
word,
面膜,2092
保湿,2074
补水,1158
洁面乳,967
男士,877
...,...
扑8048,1
扑8049,1
扑8400,1


In [11]:
all_words = g.index[(g['cnt']>4)].tolist()
len(all_words)

5363

In [12]:
all_words = list(set(all_words))
print(len(all_words))
vocab = { j:i+1 for i, j in enumerate(all_words)}
vocab["unk"] = 0

5363


In [13]:
from keras.preprocessing.sequence import pad_sequences
import numpy as np

train['name'] = train['name'].apply(lambda x:[vocab.get(i,0) for i in x])
Sens = pad_sequences(train['name'],maxlen=40)
labels = np.array(train['vector_cls'].tolist())

In [14]:
from keras.layers import Input,Embedding,LSTM,Dense,Lambda
from keras.layers.merge import dot
from keras.models import Model
from keras import backend as K
from keras.models import Sequential
from keras.layers import GlobalAveragePooling1D,BatchNormalization,Dropout
from keras.optimizers import Adam
max_features = 5500
embedding_dims = 30
maxlen = 40

model = Sequential()
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(GlobalAveragePooling1D())
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='selu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 30)            165000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 30)                0         
_________________________________________________________________
batch_normalization (BatchNo (None, 30)                120       
_________________________________________________________________
dropout (Dropout)            (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                310       
_________________________________________________________________
batch_normalization_1 (Batch (None, 10)                40        
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0

In [15]:
model.fit(Sens,labels,batch_size=100,epochs=10,validation_split=0.1)

Epoch 1/10
180/180 [==============================] - 1s 6ms/step - loss: 0.8369 - accuracy: 0.7479 - val_loss: 1.4015 - val_accuracy: 0.7100
Epoch 2/10
180/180 [==============================] - 1s 4ms/step - loss: 0.4294 - accuracy: 0.8491 - val_loss: 1.4425 - val_accuracy: 0.7100
Epoch 3/10
180/180 [==============================] - 1s 4ms/step - loss: 0.3678 - accuracy: 0.8645 - val_loss: 1.1820 - val_accuracy: 0.7470
Epoch 4/10
180/180 [==============================] - 1s 4ms/step - loss: 0.3470 - accuracy: 0.8681 - val_loss: 0.3774 - val_accuracy: 0.8625
Epoch 5/10
180/180 [==============================] - 1s 4ms/step - loss: 0.3184 - accuracy: 0.8859 - val_loss: 0.3438 - val_accuracy: 0.8730
Epoch 6/10
180/180 [==============================] - 1s 4ms/step - loss: 0.2936 - accuracy: 0.8907 - val_loss: 1.0448 - val_accuracy: 0.7555
Epoch 7/10
180/180 [==============================] - 1s 4ms/step - loss: 0.2882 - accuracy: 0.8966 - val_loss: 0.3812 - val_accuracy: 0.8930
Epoch 

In [16]:
def cat(s):
    s = np.expand_dims(s, 1)
    l = model.predict(s.T).tolist()[0]
    return clsdict[l.index(max(l))]

In [17]:
test['name'] = test['name'].apply(lambda x:seg(x, stop_words(), apply=clean_txt))

In [18]:
test['name'] = test['name'].apply(lambda x:bigram(x))

In [19]:
test['vec'] = test['name'].apply(lambda x:[vocab.get(i,0) for i in x])

In [20]:
Sens_test = pad_sequences(test['vec'],maxlen=40)

In [21]:
Sens_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 1805, 2712, 1432,    0,  782, 1482,
       3265,    0,    0,    0,    0,    0, 5170])

In [22]:
test['vec_long'] = None
for i,r in test.iterrows():
    test.at[i,'vec_long'] = Sens_test[i]

In [24]:
#test['precat'] = test['vec_long'].apply(lambda x:cat(x))

In [ ]:
mark = 0
for i,r in test.iterrows():
    if r['cls_2'] == r['precat']:
        mark += 1
print(mark/10542)